# Part 0: Setting up

## Install and start MongoDB

In [ ]:
!apt install -qq mongodb
!service mongodb start

## Download dataset and push to our mongodb 

In [ ]:
# It's already the 21st century and people are very impatient, so they use Brotli for text and Zstd for everything else.
# Reference: https://github.com/google/brotli
!apt-get install -qq brotli

In [ ]:
!wget -q https://csc14118.github.io/thuoc_raw.json.br
!wget -q https://csc14118.github.io/gia_ke_khai_raw.json.br
!wget -q https://csc14118.github.io/movies_lang.json.br 

In [ ]:
!brotli -d *.br

In [ ]:
!pip install -q pymongo

In [ ]:
import json
from pymongo import MongoClient

client = MongoClient()

# Creation of the new database
db = client['input_data']

collection_name = ["gia_ke_khai_raw", "movies_lang", "thuoc_raw"]

# Push our data to mongodb
for data in collection_name:
    collection = db[data]
    chunks = json.load(open(f'{data}.json'))
    collection.insert_many(chunks)

# Create a dummy database to test
db = client['dummy']
db['chunks'].insert_many([{'Banh xeo': 'Rat ngon'},{'Banh bao': 'Cung ngon'}])

client.list_database_names()

## Install pyspark

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q "https://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz"
!tar xf spark-3.1.1-bin-hadoop2.7.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "spark-3.1.1-bin-hadoop2.7"
import findspark
findspark.init()
findspark.find()

In [ ]:
import pyspark
print(pyspark.__version__)

## Dirty trick to connect spark to our mongodb

In industry environment, please read the docs carefully to seting up these complicated things.

In [ ]:
!rm $SPARK_HOME/jars/mongo*.jar
!rm $SPARK_HOME/jars/bson*.jar

In [ ]:
!cd $SPARK_HOME/jars && wget https://repo1.maven.org/maven2/org/mongodb/spark/mongo-spark-connector_2.12/10.1.1/mongo-spark-connector_2.12-10.1.1.jar
!cd $SPARK_HOME/jars && wget https://repo1.maven.org/maven2/org/mongodb/mongodb-driver/3.12.12/mongodb-driver-3.12.12.jar
!cd $SPARK_HOME/jars && wget https://repo1.maven.org/maven2/org/mongodb/mongo-java-driver/3.12.12/mongo-java-driver-3.12.12.jar
!cd $SPARK_HOME/jars && wget https://repo1.maven.org/maven2/org/mongodb/bson/4.6.0/bson-4.6.0.jar

In [ ]:
from pyspark.shell import spark
from pyspark import SparkContext,SparkConf

uri = "mongodb://localhost:27017/input_data"

from pyspark.sql import SparkSession

spark_jb = "org.mongodb.spark:mongo-spark-connector_2.12:10.1.1"
my_spark = SparkSession \
    .builder \
    .config("spark.executor.memory", "1g") \
    .appName("csc14112") \
    .config("spark.mongodb.read.connection.uri", uri) \
    .config("spark.mongodb.write.connection.uri", uri) \
    .getOrCreate()

In [ ]:
# Test read data from our mongo db
p = my_spark.read.format("mongodb").option("database","dummy").option("collection", "chunk").load()
p.printSchema()

In [ ]:
p.show()

# Part 1: Introduction to PySpark


In this lab assignment, we will work with a movie dataset loaded into our MongoDB at `input_data.movies_lang`. We will use PySpark RDD and DataFrame to perform the following tasks:

In [ ]:
# YOUR CODE HERE
# Read data from mongodb
data = my_spark.read.format("mongodb").option("database","input_data").option("collection", "movies_lang").load()
data_rdd = data.rdd
data.printSchema()
data.show()

### (a) Count the number of movies by country. Sort by count in decreasing order.

In [ ]:
# YOUR CODE HERE
# Group by country in data then sort by count in decreasing oder
data.groupBy("country").count().sort("count", ascending=False).show()

### (b) Return the titles of the movies produced in France.

In [ ]:
# YOUR CODE HERE
# First we get row have row.country = "FR". Then get row.title
res = data_rdd.filter(lambda row: row.country == "FR").map(lambda row: row.title)
res.collect()

### (c) Return the title of the movies of which Sofia Coppola is one of the actresses. 

In [ ]:
# YOUR CODE HERE
# Sofia Coppola is one of the actresses in actor but actors have more actor so we for loop to get row have any Sofia Coppola then get title
filtered_actor = data_rdd.filter(lambda row: any(actor.first_name == "Sofia" and actor.last_name == "Coppola" for actor in row.actors)).map(lambda row: row.title)
filtered_actor.collect()


### (d) Return the names and birth dates of the directors of movies produced in France.


In [ ]:
# YOUR CODE HERE
# First we need get row with row["country"] = "FR". Then we map first name, last name and birth_date of director
res = data_rdd.filter(lambda row: row["country"] == "FR").map(lambda row: (row["director"]["first_name"] + " " + row["director"]["last_name"], row["director"]["birth_date"]))
res.collect()

### (e) Return the average number of actors in a film.


In [ ]:
# YOUR CODE HERE
data.select("actors").rdd.map(lambda x: len(x[0])).mean()

### (f) Return the name of the actor that acted in the most movies.

In [ ]:
# YOUR CODE HERE
actor_flat = data.select("actors").rdd.flatMap(lambda x: x[0])
actor_count = actor_flat.map(lambda x: (x, 1)).reduceByKey(lambda x, y: x + y)
actor_count.sortBy(lambda x: x[1], ascending=False).take(1)

# Part 2: Real-world Data Manipulation

In this part of the lab, we will work with two collections in our MongoDB: `gia_ke_khai_raw` and `thuoc_raw` loaded at `input_data.gia_ke_khai_raw` and `input_data.thuoc_raw` respectively. We will use PySpark RDD and DataFrame to perform the following tasks:

### (a)  Read the datasets into a DataFrame and print out the schema and the number of records.

In [ ]:
# YOUR CODE HERE
data_thuoc = my_spark.read.format("mongodb").option("database","input_data").option("collection", "thuoc_raw").load()
data_gia = my_spark.read.format("mongodb").option("database","input_data").option("collection", "gia_ke_khai_raw").load()
data_thuoc.printSchema()
data_gia.printSchema()

### (b) Show all records in the `thuoc_raw` collection that have the same active pharmaceutical ingredient (API) in their `hoatChat` field as their medicine name.


Notes: In the context of medication, API stands for Active Pharmaceutical Ingredient, which is the biologically active component in a drug that produces the intended therapeutic effect. In other words, it is the chemical substance that gives a medicine its medicinal properties.

In [ ]:
# YOUR CODE HERE
data_thuoc_rdd = data_thuoc.rdd
anwser_b = data_thuoc_rdd.filter(lambda row: row.tenThuoc == row.hoatChat).map(lambda row: (row.tenThuoc,row.hoatChat))
anwser_b.collect()

### (c) Create a new DataFrame from the `thuoc_raw` collection that splits the API in the `hoatChat` field into multiple rows. For example, "paracetamol" is the API in "Paracetamol 500 mg," and "amoxicillin" is the API in various medications such as "Amogentine 500mg/125mg," "Augbactam 1g/200mg," and "Viamomentin." The resulting DataFrame should have two columns: `hoatChat` and `thuocTuongUng` as a list. After processing the data, write it back to our MongoDB at `output_data.thuocthaythe`.

In [ ]:
# YOUR CODE HERE
reduce_data = data_thuoc.select("hoatChat", "tenThuoc").rdd.flatMap(lambda x: [(i, x[1]) for i in x[0].split(",")]).toDF(["hoatChat", "thuocTuongUng"])
reduce_data.show()

# Write dataframe back to mongodb at output_data.thuocthaythe
reduce_data.write.format("mongodb").option("database","output_data").option("collection", "thuocthaythe").mode("overwrite").save()

In [ ]:
# check data in mongodb of collection output_data.thuocthaythe
df = my_spark.read.format("mongodb").option("database","output_data").option("collection", "thuocthaythe").load()
df.show()

### (d) Create new DataFrame from two mentioned above that contains  `tenThuoc`, `hoatChat`, `dongGoi`, `dvt` and `giaBan`. After process the data, write it back to our mongodb at `output_data.giathuoc`.

In [ ]:
# YOUR CODE HERE
# Create dataframe from rdd of data_thuoc and data_gia 
# - Select columns of data_thuoc: id, tenThuoc, hoatChat, dongGoi -> to dataframe df_thuoc
# - Select columns of data_gia: sdk, dvt, giaBan -> to dataframe df_gia
df_thuoc = data_thuoc.rdd.map(lambda x: (x["id"], x["tenThuoc"], x["hoatChat"], x["dongGoi"])).toDF(["id", "tenThuoc", "hoatChat", "dongGoi"])
df_gia = data_gia.rdd.map(lambda x: (x["sdk"], x["dvt"], x["giaBan"])).toDF(["sdk", "dvt", "giaBan"])

# Join df_thuoc and df_gia on id = sdk
df_thuoc_gia = df_thuoc.join(df_gia, df_thuoc.id == df_gia.sdk, "inner")

# create new dataframe from rdd of df_thuoc_gia
df = spark.createDataFrame(df_thuoc_gia.rdd, df_thuoc_gia.schema)

# choose id primary key for mongodb
df = df.withColumn("_id", df.id)

# drop column sdk, id in df_thuoc_gia
df = df.drop("sdk", "id")

# show dataframe
df.show()

# Write dataframe back to mongodb at output_data.giathuoc
df.write.format("mongodb").option("database","output_data").option("collection", "giathuoc") .mode("overwrite").save()


In [ ]:
# check data in mongodb of collection output_data.giathuoc
df = my_spark.read.format("mongodb").option("database","output_data").option("collection", "giathuoc").load()
df.show()